In [4]:
! load_ext autoreload
! autoreload 2
from diffusers import StableDiffusionPipeline, EulerAncestralDiscreteScheduler
import torch
import os
import json
import math
from itertools import groupby
from typing import Callable, Dict, List, Optional, Set, Tuple, Type, Union

import numpy as np
import PIL
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys
sys.path.append('../')
from lora_diffusion import LoraInjectedConv2d, LoraInjectedLinear, patch_pipe, tune_lora_scale
from lora_diffusion.lora import _find_modules, UNET_CROSSATTN_TARGET_REPLACE
from reg_lora.visual import visualize_images

# os.environ["DISABLE_TELEMETRY"] = 'YES'
model_id = "runwayml/stable-diffusion-v1-5"
device = torch.device("cuda:3")
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16,local_files_only=True,revision='39593d5650112b4cc580433f6b0435385882d819').to(device)
pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)

prompt = "style of <s1><s2>, baby lion"
torch.manual_seed(0)
# image = pipe(prompt, num_inference_steps=50, guidance_scale=7).images[0]

# image  # nice. diffusers are cool.

zsh:1: command not found: load_ext
zsh:1: command not found: autoreload


In [ ]:
lora_path = '/home/zhicai/poseVideo/lora-master/output_dog_Ti-Noclip_norm/lora_weight_e62_s10000.safetensors'
# lora_path = '/home/zhicai/poseVideo/lora-master/output_dog_Ti-Noclip_norm/lora_weight_e62_s10000.safetensors'
# lora_path = '../output_dog_cross+self_tR0.001_nR0.001/lora_weight_e12_s2000.safetensors'

patch_pipe(
    pipe,
    lora_path,
    patch_text=False,
    patch_ti=True,
    patch_unet=True,
    filter_crossattn_str = 'cross+self'
)

prompts = 'photo of a <krk1> dog swimming in the pool' 
tune_lora_scale(pipe.unet, 0)
torch.manual_seed(0)
img = pipe([prompts] * 4, num_inference_steps=50, guidance_scale=7).images
visualize_images(img,outpath=' ',nrow = 5, save = False)

In [9]:
import copy
import glob
import re
# '../output_dog_cross+self_tR0.001_nR0.001'
_lora_path = '../output/output_dog_Ti-clip_norm_3e-5'
prompts = ['photo of a <krk1> dog swimmking in the pool'] 
images = []
bs = 4
cnt = 0
_files = os.listdir(_lora_path)
pattern = r'lora_weight_e\d+_s\d{3,}.safetensors'
files = [f for f in _files if re.match(pattern, f)]

for i, path in enumerate(sorted(files, key = lambda x: int(re.match(r'.*e([0-9]+).*', x.split('/')[-1])[1]), reverse=False)):
    if i % 10 ==0:
        cnt +=1 
        lora_path = os.path.join(_lora_path, path)
        print(lora_path)
        pipe_copy = copy.deepcopy(
            pipe)
        torch.manual_seed(0)
        patch_pipe(
            pipe_copy,
            lora_path,
            patch_text=False,
            patch_ti=True,
            patch_unet=True,
            filter_crossattn_str = 'cross+self'
        )
        # pipe.unet
        tune_lora_scale(pipe_copy.unet, 0.5)
        # tune_lora_scale(pipe_copy.text_encoder, 1)
        for prompt in prompts:
            prompt = [prompt]*bs
            img = pipe_copy(prompt = prompt, num_inference_steps=50, guidance_scale=6).images
            images.extend(img)
if len(images) > 0:
    visualize_images(images,outpath='photo_of_krk1_norm_dog', nrow=4, show=False,save=True)
else:
    print('no images')


../output/output_dog_Ti-clip_norm_3e-5/lora_weight_e1_s200.safetensors
<krk1>


100%|██████████| 50/50 [00:11<00:00,  4.52it/s]


../output/output_dog_Ti-clip_norm_3e-5/lora_weight_e13_s2200.safetensors
<krk1>


100%|██████████| 50/50 [00:11<00:00,  4.51it/s]


../output/output_dog_Ti-clip_norm_3e-5/lora_weight_e26_s4200.safetensors
<krk1>


100%|██████████| 50/50 [00:11<00:00,  4.51it/s]


../output/output_dog_Ti-clip_norm_3e-5/lora_weight_e38_s6200.safetensors
<krk1>


100%|██████████| 50/50 [00:11<00:00,  4.51it/s]


../output/output_dog_Ti-clip_norm_3e-5/lora_weight_e51_s8200.safetensors
<krk1>


100%|██████████| 50/50 [00:11<00:00,  4.52it/s]


In [ ]:
import copy
import glob
import re
# '../output_dog_cross+self_tR0.001_nR0.001'
_lora_path = '../output_dog_cross+self_tR0.001_Scale-nR0.01'
prompts = ['<krk> dog in a construction outfit','<krk> dog in times square'] 
images = []
bs = 2
cnt = 0
_files = os.listdir(_lora_path)
pattern = r'lora_weight_e\d+_s\d{4,}.safetensors'
files = [f for f in _files if re.match(pattern, f)]

for path in sorted(files, key = lambda x: int(re.match(r'.*e([0-9]+).*', x.split('/')[-1])[1]), reverse=True):
    cnt +=1 
    lora_path = os.path.join(_lora_path, path)
    print(lora_path)
    pipe_copy = copy.deepcopy(
        pipe)
    torch.manual_seed(0)
    patch_pipe(
        pipe_copy,
        lora_path,
        patch_text=False,
        patch_ti=True,
        patch_unet=True,
        filter_crossattn_str = 'cross+self'
    )
    # pipe.unet
    tune_lora_scale(pipe_copy.unet, 1)
    # tune_lora_scale(pipe_copy.text_encoder, 1)
    for prompt in prompts:
        prompt = [prompt]*bs
        img = pipe_copy(prompt = prompt, num_inference_steps=50, guidance_scale=6).images
        images.extend(img)
    if cnt >=3 :
        break
if len(images) > 0:
    visualize_images(images,outpath='photo_of_krk_dog.jpg', nrow=4, save=False)
else:
    print('no images')
